In [1]:
import os
import torch
import transformers
import logging as log
import re

/home/anshuman/workspace/projects/FluxFrame-server/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MAX_INPUT_TOKEN_LENGTH = 4000
MAX_TOKEN = 3840
log.basicConfig(level=log.INFO, format=" %(levelname)s %(message)s")
history = []

In [3]:
bnb_config = transformers.BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

model = transformers.AutoModelForCausalLM.from_pretrained(
    "codellama/CodeLlama-13b-Instruct-hf",
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map="auto",
    # load_in_8bit_fp32_cpu_offload=True
)
model.eval()


 INFO We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
Loading checkpoint shards: 100%|██████████| 3/3 [00:07<00:00,  2.43s/it]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32016, 5120)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear4bit(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm

In [4]:
# SYSTEM_PROMPT = """
# You are a helpful experienced web developer and designer with a deep knowledge of HTML, CSS, Web-development and software UI design. Follow all the instructions provided, but can also give more features.\nDo not give multiple answers or repeat things. Make beautiful designs using CSS based on your creativity.\nPlease don't provide wrong code. \n Give a complete solution with maximum number of features. Code practice: Give all the CSS inside <style> tag mapped with class names and id. Do not link any files to head link tag. You may also write necessary correct javascript by creating functions inside <script> tag, use buttons calling function and don't use form tag. Always check if all the components are correctly written and mapped.\
# """

# PROMPT = "A personal portfolio with a navbar with big text of 'About' and 'Blog' in center The page should be divided into 2 parts vertically, left side should have a profile pic and right side should have a short description about the person. The page should have a footer with social media icons."

In [10]:
SYSTEM_PROMPT = """
You are a helpful experienced web developer and designer with a deep knowledge of React, TailwindCSS, JavaScript, Web-development and Software UI design. Follow all the instructions provided, but can also give more features.\nDo not give multiple answers or repeat things. Make beautiful designs using TailwindCSS based on your creativity.\nPlease don't provide wrong code. \n Give a complete solution with maximum number of features. 
Code practice: Give only the JSX code, using tailwindCSS in its className. Do not give globals.css CSS code,just give tailwindCSS class in className. You may also write necessary correct javascript by creating functions inside <script> tag, use buttons calling function and don't use form tag. Always check if all the components are correctly written and mapped.\
"""

PROMPT = "A personal portfolio navbar with big text of 'About' and 'Blog' in center. The design should minimilistic asthetic dark."

In [6]:
def get_prompt(message: str,system_prompt: str, history_code: str = "<></>", infilling=False) -> str:
    # texts = [f'<s><<SYS>>\n{system_prompt}\n<</SYS>>\n\n[INST]{message}\n[/INST]\n<!DOCTYPE html> <html><FILL_ME></html></s>']
    texts =  f"<<SYS>>\n{system_prompt}\n<</SYS>>\n[INST]{message}\n[/INST]\n" 
    texts+="import React from 'react';\nimport '../styles/globals.css';\n\nfunction Component(){\n<FILL_ME> \n}\nexport default Component;"
    
    log.info(f'Prompt: {texts}')
    history = texts
    return ''.join(texts)




In [7]:
def get_html_block(filling):
    html_pattern = r'<html>(.*?)</html>'
    matches = re.findall(html_pattern, filling, re.DOTALL)
    
    if matches:
        return matches[0]
    else:
        return filling

def prepare_react_code(filling):
    start_marker = 'function Component(){'
    end_marker = 'export default Component;'
    
    start_index = filling.find(start_marker)
    end_index = filling.find(end_marker, start_index)

    print(start_index, end_index)

    if start_index != -1 and end_index != -1 and end_index > start_index:
        content = filling[start_index + len(start_marker):end_index].strip()
        filling =  content
    else:
        filling = 'return() '
        
    code = '''
        import React from "react";
        import '../styles/globals.css'
             
          function Component(){
          
          '''+filling+'''
          
          export default Component;
        '''
    return code

def write_to_file(filename, content):
    with open(filename, 'w') as file:
        file.write(content)
        file.close()

def generate(prompt, model, device):
    tokenizer = transformers.CodeLlamaTokenizer.from_pretrained(
        "codellama/CodeLlama-13b-Instruct-hf"
    )
    processed_prompt = get_prompt(prompt,SYSTEM_PROMPT)
    input_ids = tokenizer(processed_prompt, return_tensors="pt", device=device)["input_ids"]
    generated_ids = model.generate(input_ids.to(device), max_new_tokens=MAX_TOKEN)
    filling = tokenizer.batch_decode(generated_ids[:, input_ids.shape[1] :], skip_special_tokens=True)[0]

    log.info(f'Filling: {filling}')
    gen_code = f'{prepare_react_code(filling)}'
    write_to_file('./component.js', gen_code)
    return gen_code

In [8]:
filling = '''

import React from "react";
    
function Component(){

return (
    <div className="bg-black text-white">
      <div className="flex justify-center items-center">
        <div className="text-4xl font-bold">
          <a href="#" className="hover:text-white">About</a>
        </div>
        <div className="text-4xl font-bold">
          <a href="#" className="hover:text-white">Blog</a>
        </div>
      </div>
    </div>
  );
}

export default Component;
iv className="bg-black text-white">
      <div className="flex justify-center items-center">
        <div className="text-4xl font-bold">
          <a href="#" className="hover:text-white">About</a>
        </div>
        <div className="text-4xl font-bold">
          <a href="#" className="hover:text-white">Blog</a>
        </div>
      </div>
    </div>
  );
}

export default Component;

import React from "react";
    
function Component(){

return (
    <div className="bg-black text-white">
      <div className="flex justify-center items-center">
        <div className="text-4xl font-bold">
          <a href="#" className="hover:text-white">About</a>
        </div>
        <div className="text-4xl font-bold">
          <a href="#" className="hover:text-white">Blog</a>
        </div>
      </div>
    </div>

'''

def prepare_code(filling):
    pattern = r'function\s+Component\(\)\s*{([\s\S]+?)\n}\n\nexport\s+default\s+Component;'
    match = re.search(pattern, filling)

    if match:
        filling = match.group(1)
    else:
        filling = 'return() '
        
        
    code = '''
        import React from "react";
        import '../styles/globals.css'
             
          function Component(){
          
          '''+filling+'''
          
          export default Component;
        '''
    return code

print(prepare_code(filling))



        import React from "react";
        import '../styles/globals.css'
             
          function Component(){
          
          

return (
    <div className="bg-black text-white">
      <div className="flex justify-center items-center">
        <div className="text-4xl font-bold">
          <a href="#" className="hover:text-white">About</a>
        </div>
        <div className="text-4xl font-bold">
          <a href="#" className="hover:text-white">Blog</a>
        </div>
      </div>
    </div>
  );
          
          export default Component;
        


In [9]:

output = generate(PROMPT, model, device='cuda')

print(output)

 INFO Prompt: <<SYS>>

You are a helpful experienced web developer and designer with a deep knowledge of React, TailwindCSS, JavaScript, Web-development and Software UI design. Follow all the instructions provided, but can also give more features.
Do not give multiple answers or repeat things. Make beautiful designs using TailwindCSS based on your creativity.
Please don't provide wrong code. 
 Give a complete solution with maximum number of features. 
Code practice: Give only the JSX code, using tailwindCSS in its className. Dont write CSS code, give tailwindCSS class in className. You may also write necessary correct javascript by creating functions inside <script> tag, use buttons calling function and don't use form tag. Always check if all the components are correctly written and mapped.
<</SYS>>
[INST]A personal portfolio navbar with big text of 'About' and 'Blog' in center. The design should minimilistic asthetic dark.
[/INST]
import React from 'react';
import '../styles/globals.c

-1 -1

        import React from "react";
        import '../styles/globals.css'
             
          function Component(){
          
          return() 
          
          export default Component;
        
